# Experimentation

In [1]:
from rstt import Ranking
from rstt.stypes import SPlayer
import pandas as pd
from scipy import stats

def evaluate(data: pd.DataFrame, reference: Ranking, trained: Ranking, population: list[SPlayer], percentiles: list[int]=[5, 10, 50]):
    row = {}
    
    # rank correlation
    row['KendallTau'] = stats.kendalltau(reference[population], trained[population]).statistic

    # rank difference
    rank_diffs = [reference[p]- trained[p] for p in population]
    row['Overrate'] = abs(min(rank_diffs))
    row['Underrate'] = max(rank_diffs)

    # accuracy
    for percentile in percentiles:
        threshold = max(int(len(population) // (100/percentile)), 1)
        row[f'{percentile}pr accuracy'] = len(set(reference[:threshold]).intersection(set(trained[:threshold]))) / threshold

    data = pd.concat([data, pd.DataFrame([row])], axis=0, ignore_index=True)
    return data

In [2]:
from rstt import BasicPlayer, BasicElo, LogSolver, BTRanking, RoundRobin

from simulation import settings, experiments, metrics

# parameters
samples = 100
NBPlayers = 60

# data 
COLUMNS = ['KendallTau', 'Underrate', 'Overrate', '5pr accuracy', '10pr accuracy', '50pr accuracy']
RESULTS: dict[str, pd.DataFrame] = {}

exp_name = 'FullRoundRobin'
RESULTS[exp_name] = pd.DataFrame(columns=COLUMNS, dtype='float')
for i in range(samples):
    # test ranking
    elo = BasicElo(f'Elo-{exp_name}')
    
    # simulation model
    model = settings.baseModel(region=1, nb=NBPlayers)
    
    # data set production
    data = experiments.regional_round_robins(f'{exp_name}-Round-Robin',
                                             seeding=model.groundtruth,
                                             population=[model.ecosystem.teams()],
                                             solver=model.solver)
    # elo update
    elo.update(**data)

    # analysis
    RESULTS[exp_name] = evaluate(RESULTS[exp_name], model.groundtruth, elo, model.groundtruth.players())

# show summary
RESULTS[exp_name].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.0000,100.000000,100.000000,100.000000,100.000000
mean,0.903808,8.2000,8.400000,0.816667,0.880000,0.951000
std,0.014707,2.0695,2.283007,0.197487,0.111313,0.031589
min,0.859887,4.0000,5.000000,0.333333,0.500000,0.866667
25%,0.894633,7.0000,7.000000,0.666667,0.833333,0.933333
50%,0.903955,8.0000,8.000000,0.833333,0.833333,0.950000
75%,0.912994,9.0000,9.000000,1.000000,1.000000,0.966667
max,0.934463,14.0000,16.000000,1.000000,1.000000,1.000000


In [3]:
from project import scene

exp_name = 'Regions'

# store metrics
for region in scene.Region:
    RESULTS[f'{exp_name}-{region}'] = pd.DataFrame(columns=COLUMNS, dtype='float')
RESULTS[exp_name]  = pd.DataFrame(columns=COLUMNS, dtype='float')

for i in range(samples):
    elo2 = BasicElo(f'Elo-{exp_name}')
    
    model = settings.baseModel(len(scene.Region), 10)
    data = experiments.regional_round_robins(f'{exp_name}-Round-Robin',
                                             seeding=model.groundtruth,
                                             population=[model.ecosystem.teams(region) for region in model.ecosystem.regions()],
                                             solver=model.solver)
    elo2.update(**data)
    
    # analysis
    for region in model.ecosystem.regions():
        RESULTS[f'{exp_name}-{region}'] = evaluate(RESULTS[f'{exp_name}-{region}'], model.groundtruth, elo2, model.ecosystem.teams(region))
    RESULTS[exp_name]  = evaluate(RESULTS[exp_name], model.groundtruth, elo2, model.ecosystem.teams())

# show summary
RESULTS[exp_name].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.708554,20.700000,20.560000,0.390000,0.591667,0.865333
std,0.044987,3.935079,4.416041,0.259737,0.148647,0.044676
min,0.567232,13.000000,13.000000,0.000000,0.333333,0.766667
25%,0.676836,18.000000,17.000000,0.333333,0.500000,0.833333
50%,0.711299,20.000000,20.000000,0.333333,0.666667,0.866667
75%,0.741808,23.000000,23.000000,0.666667,0.666667,0.900000
max,0.811299,33.000000,34.000000,1.000000,1.000000,0.966667


In [4]:
exp_name = 'SkilledRegions'

for region in scene.Region:
    RESULTS[f'{exp_name}-{region}'] = pd.DataFrame(columns=COLUMNS, dtype='float')
RESULTS[exp_name]  = pd.DataFrame(columns=COLUMNS, dtype='float')

for i in range(samples):
    # test ranking
    elo3 = BasicElo(f'Elo-{exp_name}')
    
    # simulation model
    model = settings.regionalSkills(region=len(scene.Region),
                                    nb=10,
                                    mus=[val for val in range(750, 2500, 250)],
                                    sigmas=[300  for _ in scene.Region])
    
    # data set production
    data = experiments.regional_round_robins(f'{exp_name}-Round-Robin',
                                             seeding=model.groundtruth,
                                             population=[model.ecosystem.teams(region) for region in model.ecosystem.regions()],
                                             solver=model.solver)
    # elo update
    elo3.update(**data)

    # analysis
    for region in scene.Region:
        RESULTS[f'{exp_name}-{region}'] = evaluate(RESULTS[f'{exp_name}-{region}'], model.groundtruth, elo3, model.ecosystem.teams(region))
    RESULTS[exp_name]  = evaluate(RESULTS[exp_name], model.groundtruth, elo3, model.groundtruth.players())

# show summary
RESULTS[exp_name].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.324757,37.900000,38.680000,0.263333,0.345000,0.636667
std,0.041202,3.870374,4.385191,0.208113,0.161111,0.048432
min,0.205650,29.000000,29.000000,0.000000,0.000000,0.500000
25%,0.296045,34.750000,36.000000,0.000000,0.166667,0.600000
50%,0.322034,38.000000,39.000000,0.333333,0.333333,0.633333
75%,0.357062,41.000000,42.000000,0.333333,0.500000,0.666667
max,0.398870,45.000000,51.000000,0.666667,0.666667,0.766667


In [5]:
RESULTS[f'{exp_name}-{scene.Region.LEC}'].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.00000,100.000000,100.00000,100.00000,100.000000
mean,0.714667,20.20000,9.240000,0.21000,0.21000,0.308000
std,0.113211,5.47169,4.811854,0.40936,0.40936,0.159342
min,0.422222,7.00000,0.000000,0.00000,0.00000,0.000000
25%,0.644444,16.00000,6.000000,0.00000,0.00000,0.200000
50%,0.733333,20.00000,9.000000,0.00000,0.00000,0.400000
75%,0.822222,24.00000,13.000000,0.00000,0.00000,0.400000
max,0.955556,32.00000,22.000000,1.00000,1.00000,0.600000
